# Graphito

TO DO: Copiar template y analisis de grafazo

```
# "que cuatrimestre armo?" -> Graphito
---

# Vamos a armar un graphito: Analizar "gente que se haya armado cuatris parecidos al mio" -> Ahora tenemos factor temporal "local": No me importa que haya sido en el 2022C1, pero si me importa que las materias que curse el mismo cuatri esten en el mismo bucket
# nodos -> usuarios
# flechas -> compartir N materias el en mismo cuatri
# peso -> Cuantas materias compartimos (2, 3, 4 o 5)


---
# "que tan recursante sos? // que tan social ;) sos? "  
Tenemos ganas de hacer un grafo en el que no sea multigrafo ni sea con peso, ya que eran los grafos que mas abundaban en los ejemplos y en las posibilidades a lo largo de la materia.
# nodos -> alumnes
# flechas -> cursar al menos 1 vez la misma materia, el mismo cuatrimetre
```

In [ ]:
import pandas as pd

df = pd.read_pickle('fiuba-map-data.pickle')
df.sample(3)

In [ ]:
df_rel = df[['Padron', 'materia_id', 'materia_cuatrimestre']].copy()

# Sacamos a aquellos que no ordenan su carrera por cuatrimestres
df_rel.dropna(subset=['materia_cuatrimestre'], inplace=True)

df_rel.sample(3)

In [ ]:
# Juntamos el grafo con si mismo para tener la similiritud entre cada par de padrones
df_simil = pd.merge(df_rel, df_rel, on=['materia_id', 'materia_cuatrimestre'])
df_simil = df_simil[df_simil['Padron_x'] != df_simil['Padron_y']]

df_simil['Padron_min'] = df_simil[['Padron_x', 'Padron_y']].min(axis=1)
df_simil['Padron_max'] = df_simil[['Padron_x', 'Padron_y']].max(axis=1)
df_simil = df_simil.drop_duplicates(['Padron_min', 'Padron_max', 'materia_id']).reset_index()

df_simil = df_simil[['Padron_x', 'Padron_y', 'materia_id', 'materia_cuatrimestre']]
df_simil.rename(columns = {'Padron_x':'src_padron', 'Padron_y':'dst_padron'}, inplace = True)
display(df_simil.sample(3))
df_simil.shape

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt

G = nx.from_pandas_edgelist(df_simil, 
                            source='src_padron', 
                            target='dst_padron',
                            create_using=nx.Graph())

nx.set_node_attributes(G, df_simil.to_dict('index'))

print(G)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.cm as cm

def plot_centrality(centrality, g, size = 1):
    cent = np.fromiter(centrality.values(), float)
    sizes = [v * 500 * size for v in centrality.values()]
    normalize = mcolors.Normalize(vmin=cent.min(), vmax=cent.max())
    colormap = cm.YlOrRd
    
    plt.figure(figsize=(10, 10))
    nx.draw(g, node_size=sizes, node_color=sizes, cmap=colormap, width = 0.01, with_labels = True, font_size= 10, pos=nx.spring_layout(g, k = 1.5))
    plt.show()

In [ ]:
k_centrality = list(nx.k_core(G, 25))
k_graph = G.subgraph(k_centrality)
plot_centrality(nx.degree_centrality(k_graph), k_graph)

In [ ]:
#ahre dios que horrible, a seguir iterando ideas

In [ ]:
# Capaz algo asi? mmm hay que juntarse y debatir salu3
# from graphrole import RecursiveFeatureExtractor, RoleExtractor
# import networkx as nx
# import matplotlib.pyplot as plt

# def extract_roles_and_plot(G, title='',save = False ,file_name='', big=False):
#     feature_extractor = RecursiveFeatureExtractor(G)
#     features = feature_extractor.extract_features()
#     role_extractor = RoleExtractor(n_roles=None)
#     role_extractor.extract_role_factors(features)
  
#     available_colors = {'role_0':'#9B59B6', 'role_1':'#58D68D', 'role_2':'#AED6F1', 'role_3':'#F4D03F', 'role_4':'#00aa00','role_5':'#aaaa00','role_6':'#aa0000' , 'role_7':'#0000aa'}
    
#     colors = [available_colors[role_extractor.roles[node]] for node in G.nodes()]
    
#     pos = nx.kamada_kawai_layout(G)
#     plt.figure(figsize=(15,15))
#     plt.title(title)

#     nx.draw_networkx(G, pos, nodelist=G.nodes(), node_color=colors, alpha=0.7, node_size=200, linewidths=0.5, with_labels=True, width= 0.2, font_size = 9)